In [50]:
import weaviate

client = weaviate.connect_to_local()

print(client.is_live())

True


In [31]:
import weaviate

client = weaviate.connect_to_local()
client.collections.delete("Article")
print(client.collections.list_all(simple=False))
client.close()

{}


/var/folders/kg/fmxh64rs0c781kf4dqw14h3r0000gn/T/ipykernel_32589/2215080610.py:3: ResourceWarning: unclosed <socket.socket fd=82, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 56233), raddr=('127.0.0.1', 8080)>
  client = weaviate.connect_to_local()


In [32]:
import weaviate
import weaviate.classes as wvc

client = weaviate.connect_to_local()

try:
    client.collections.create(
        name="Article",
        vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_transformers(),  # If set to "none" you must always provide vectors yourself. Could be any other "text2vec-*" also.
        generative_config=wvc.config.Configure.Generative.anyscale(model="Llama-2-7b-chat-hf"),
        description="A written text, for example a news article or blog post",
        properties=[
            wvc.config.Property(
                data_type=wvc.config.DataType.TEXT,
                description="Category of the article",
                name="category",
            ),
            wvc.config.Property(
                data_type=wvc.config.DataType.TEXT,
                description="The content of the article",
                name="content",
            ),
        ]
    )

finally:
    client.close()

In [33]:
from datasets import load_dataset, ClassLabel
# Load the Hugging Face dataset
dataset = load_dataset("ag_news")
# Define the label names
label_names = ["World", "Sports", "Business", "Sci/Tech"]
label_feature = ClassLabel(names=label_names)

In [55]:
import weaviate
from tqdm import tqdm


with weaviate.connect_to_local(headers={
        "X-Anyscale-Api-Key": "esecret_u6s5g6fenvvrlwtd5tnf9cgv83"  # Replace with your inference API key
         }) as client:
    questions = client.collections.get("Article")
    for d in tqdm(dataset['train']):
        # Map the integer label to its corresponding string name
        label_name = label_feature.int2str(d["label"])
        
        questions.data.insert({
            "content": d["text"],
            "category": label_name,
        })

RemoteProtocolError: Server disconnected without sending a response.

In [12]:
import weaviate
import os 
os.environ["ANYSCALE_APIKEY"] = "esecret_pt7pwklepatz6w5dt7eflrharn"
prompt = "Convert this quiz question: {question} and answer: {answer} into a trivia tweet."
with weaviate.connect_to_local(
    port=8080,
    grpc_port=50051,
    headers={
        "X-AnyScale-Api-Key": os.environ["ANYSCALE_APIKEY"]  # Replace with your inference API key
    }
) as client:

        print(client.is_ready())
        questions = client.collections.get("Article")
        response = questions.generate.near_text(
                query="News from USA",
                limit=4,
                single_prompt="Explain {answer} as you might to a five-year-old."
        )
        print(response.objects[0].properties)
        #print(response.objects[0].generated)


True


WeaviateQueryError: Query call with protocol GRPC search failed with message No results for generative search despite a search request. Is a generative module enabled?.

In [8]:
response

GenerativeReturn(objects=[GenerativeObject(uuid=_WeaviateUUIDInt('9249813d-f7f8-49f9-98b1-618019196abb'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'content': 'Wen Ho Lee Reporters Held in Contempt WASHINGTON - A federal judge held five reporters in contempt Wednesday for refusing to identify their sources for stories about Wen Ho Lee, a former nuclear weapons scientist once suspected of spying.    U.S...', 'category': 'World'}, references=None, vector={}, collection='Article', generated=None), GenerativeObject(uuid=_WeaviateUUIDInt('ad33a7a1-b640-441c-979f-a275645f997d'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'content': 'Kidnappers lift death threat on US journalist An aide to Shiite cleric Muqtada al-Sadr

In [9]:
response.objects[0].generated